In [1]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
from utils import time, data_processor, constants

In [2]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu_27_countries = constants.eu
balkans = constants.balkans
balkans

['BA', 'KV', 'AL', 'RS', 'ME']

In [3]:
KEY_FINANCIALS_DETAILED_INDUSTRY_PATH = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_BANKSS_PATH = "..\\data_raw\\key_financials_detailed\\banks_global_financials_and_ratios_eur"
KEY_FINANCIALS_DETAILED_INSURANCES_PATH = "..\\data_raw\\key_financials_detailed\\insurances_global_financials_and_ratios_eur"

In [4]:
key_financials_detail_path = "..\\..\\data_raw\\key_financials_detailed\\industry_global_financials_and_ratios_eur\\"

def fetch_data_for_country(country_iso_code):
    query = f"""
    SELECT 
        bvd_id_number,
        operating_revenue_turnover_,
        number_of_employees,
        costs_of_goods_sold,
        material_costs,
        added_value,
        EXTRACT(YEAR FROM closing_date) AS year  
    FROM 
        parquet_scan('{key_financials_detail_path}*.parquet')
    WHERE
        CAST(bvd_id_number AS STRING) LIKE '{country_iso_code}%'
    ORDER BY 
        bvd_id_number, year
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()

    return df


In [5]:
# TEST
# it = fetch_data_for_country("IT")
# ba = fetch_data_for_country("BA")
# cs = fetch_data_for_country("CS")
# mk = fetch_data_for_country("MK")
# xk = fetch_data_for_country("XK")
# it = fetch_data_for_country("IT")


In [ ]:
import os

output_path = "..\\..\\data_processed\\key_financials_detailed_processed\\"

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

for country in ['KV']:
    df = fetch_data_for_country(country)
    df.to_parquet(f"{output_path}key_financials_detailed_{country}.parquet")
    print(f"Saved: {output_path}key_financials_detailed_{country}.parquet")


In [ ]:
df

,bvd_id_number,operating_revenue_turnover_,number_of_employees,costs_of_goods_sold,material_costs,added_value,year
0,KV30014049,1300.0,NaN,NaN,NaN,NaN,2013
1,KV30048296,1650.0,NaN,NaN,NaN,NaN,2013
2,KV30048881,1100.0,NaN,NaN,NaN,NaN,2013
3,KV30049080,1280.0,NaN,NaN,NaN,NaN,2013
4,KV30049862,2000.0,NaN,NaN,NaN,NaN,2013
...,...,...,...,...,...,...,...
1232794,KVRS20165995,250657.0,4.0,NaN,169275.0,52846.0,2008
1232795,KVRS20289597,143543.0,2.0,NaN,69300.0,NaN,2007
1232796,KVRS20289597,638832.0,4.0,NaN,436428.0,49259.0,2008
1232797,KVRS20289597,79404.0,2.0,NaN,44919.0,NaN,2009


In [6]:
## TEST 

# firmographics_ita = pd.read_parquet("..\\..\\data_processed\\firmographics_processed\\firmographics_IT.parquet")

In [10]:
# merged_df = pd.merge(
#     firmographics_ita,
#     it,
#     left_on="bvd_id_number",
#     right_on="bvd_id_number",
#     how="outer",         
#     indicator=True       
# )


In [11]:
merged_df.value_counts("_merge")

_merge
both          75223744
left_only      1280343
right_only      151814
Name: count, dtype: int64